
## Welcome

This is material for the **Instrumental Variables** chapter in Scott Cunningham's book, [Causal Inference: The Mixtape.](https://mixtape.scunning.com/)

<img src="https://mixtape.scunning.com/images/cover.jpg" alt="Causal Inference: The Mixtape" style="width: 200px;"/>

[View the Textbook](https://mixtape.scunning.com/)


### Packages needed

The first thing you need to do is install a few packages to make sure everything runs:

In [ ]:
%load_ext rpy2.ipython

In [ ]:
%%R

install.packages("tidyverse")
install.packages("cli")
install.packages("haven")
install.packages("rmarkdown")
install.packages("learnr")
install.packages("haven")
install.packages("estimatr")
install.packages("stargazer")

# For this chapter only
install.packages("AER")
install.packages("lfe")
install.packages("SteinIV")

### Load

In [ ]:
%%R

library(learnr)
library(haven)
library(tidyverse)
library(estimatr)
library(stargazer)

# This chapter only
library(AER)
library(lfe)
library(SteinIV)

# 10 minute code time limit
options(tutorial.exercise.timelimit = 600)

# read_data function
read_data <- function(df) {
  full_path <- paste0("https://raw.github.com/scunning1975/mixtape/master/", df)
  return(haven::read_dta(full_path))
}

## Card

In [ ]:
%%R

card <- read_data("card.dta")

#OLS
ols_reg <- lm(lwage ~ educ + exper + black + south + married + smsa, 
              data = card)

cli::cli_h1("OLS")
summary(ols_reg)

#2SLS
iv_reg = ivreg(lwage ~ educ + exper + black + south + married + smsa 
               | nearc4 + exper + black + south + married + smsa, 
               data = card)

cli::cli_h1("2SLS")
summary(iv_reg)



#### Questions
- Interpret the coefficient on education when we used OLS versus when used 2SLS. 
- How does the estimated effect of education change when instrumenting with being close to a 4-year college?  That is, does the coefficient get larger or smaller compared to OLS?
- If the only source of bias in our OLS regression was omitted heterogeneous ability, then will 2SLS be larger, smaller or the same as OLS estimate?  Why/why not?   
- Is the finding of the causal effect of educating when using 2SLS, when compared to the estimate using OLS, consistent with ability bias?  What else do you think may be going on and why?
- What sorts of individuals will go to college regardless of whether a college is near them?  What sorts of individuals will never go to a college even if one is near them?  And what sorts of people will go to a college if one is near them but won't go to college if it is not near them?




## JIVE

In [ ]:
%%R

judge <- read_data("judge_fe.dta")

#grouped variable names from the data set
judge_pre <- judge %>% 
  select(starts_with("judge_")) %>% 
  colnames() %>% 
  subset(., . != "judge_pre_8") %>% # remove one for colinearity
  paste(., collapse = " + ")

demo <- judge %>% 
  select(black, age, male, white) %>% 
  colnames() %>% 
  paste(., collapse = " + ")

off <- judge %>% 
  select(fel, mis, sum, F1, F2, F3, M1, M2, M3, M) %>% 
  colnames() %>% 
  paste(., collapse = " + ")

prior <- judge %>% 
  select(priorCases, priorWI5, prior_felChar, 
         prior_guilt, onePrior, threePriors) %>% 
  colnames() %>% 
  paste(., collapse = " + ")

control2 <- judge %>%
  mutate(bailDate = as.numeric(bailDate)) %>% 
  select(day, day2, bailDate, 
         t1, t2, t3, t4, t5) %>% # all but one time period for colinearity
  colnames() %>% 
  paste(., collapse = " + ")

#formulas used in the OLS
min_formula <- as.formula(paste("guilt ~ jail3 + ", control2))
max_formula <- as.formula(paste("guilt ~ jail3 + possess + robbery + DUI1st + drugSell + aggAss",
                                demo, prior, off, control2, sep = " + "))

#max variables and min variables
min_ols <- lm_robust(min_formula, data = judge)
max_ols <- lm_robust(max_formula, data = judge)

cli::cli_h1("OLS")
texreg::knitreg(list(min_ols, max_ols))


#--- Instrumental Variables Estimations
#-- 2sls main results
#- Min and Max Control formulas
min_formula <- as.formula(paste("guilt ~ ", control2, " | 0 | (jail3 ~ 0 +", judge_pre, ")"))
max_formula <- as.formula(paste("guilt ~", demo, "+ possess +", prior, "+ robbery +", 
                                off, "+ DUI1st +", control2, "+ drugSell + aggAss | 0 | (jail3 ~ 0 +", judge_pre, ")"))

#2sls for min and max

min_iv <- lfe::felm(min_formula, data = judge)
max_iv <- lfe::felm(max_formula, data = judge)

cli::cli_h1("IV")
texreg::knitreg(list(min_iv, max_iv))



#-- JIVE main results
#- minimum controls
y <- judge %>%
  pull(guilt)

X_min <- judge %>%
  mutate(bailDate = as.numeric(bailDate)) %>%
  select(jail3, day, day2, t1, t2, t3, t4, t5, bailDate) %>%
  model.matrix(data = .,~.)

Z_min <- judge %>%
  mutate(bailDate = as.numeric(bailDate)) %>%
  select(-judge_pre_8) %>%
  select(starts_with("judge_pre"), day, day2, t1, t2, t3, t4, t5, bailDate) %>%
  model.matrix(data = .,~.)

cli::cli_h1("JIVE")
jive.est(y = y, X = X_min, Z = Z_min)

#- maximum controls
X_max <- judge %>%
  mutate(bailDate = as.numeric(bailDate)) %>%
  select(jail3, white, age, male, black,
         possess, robbery, prior_guilt,
         prior_guilt, onePrior, priorWI5, prior_felChar, priorCases,
         DUI1st, drugSell, aggAss, fel, mis, sum,
         threePriors,
         F1, F2, F3,
         M, M1, M2, M3,
         day, day2, bailDate, 
         t1, t2, t3, t4, t5) %>%
  model.matrix(data = .,~.)

Z_max <- judge %>%
  mutate(bailDate = as.numeric(bailDate)) %>%
  select(-judge_pre_8) %>%
  select(starts_with("judge_pre"), white, age, male, black,
         possess, robbery, prior_guilt,
         prior_guilt, onePrior, priorWI5, prior_felChar, priorCases,
         DUI1st, drugSell, aggAss, fel, mis, sum,
         threePriors,
         F1, F2, F3,
         M, M1, M2, M3,
         day, day2, bailDate, 
         t1, t2, t3, t4, t5) %>%
  model.matrix(data = .,~.)

jive.est(y = y, X = X_max, Z = Z_max)


#### QUESTION
- Interpret the coefficient on our two IV estimators?  How do they compare to our OLS estimate?
- What is your conclusion about the effect that cash bail has on adjudication?  Speculate about the channels by which cash bail has this effect. 
- Describe the four sub-populations (e.g., always takers, never takers, defiers and compliers) in the context of Stevenson's study.
- Discuss the plausibility of each of the 5 IV assumptions in Stevenson's case.  
- Draw a DAG that must be true for Stevenson's JIVE estimates to be consistent?  Which assumptions are contained in this DAG and which ones are not easily visualized? 
- Assume judge A is stricter than judge B.  Monotonicity requires that if judge B sets a lower bail amount for that individual, then judge A will always set a higher for that individual hypothetically than judge B.  Provide some examples where you think this may be violated.